In [1]:
import os
from cht_tiling.tiled_web_map import *

In [2]:
path = r"c:\delftdashboard\data\bathymetry\gebco_2024"
name = "gebco_2024"

# Initialize TiledWebMap instance
ds = TiledWebMap(path, name)

In [3]:
from cht_tiling.utils import lat_lon_to_webmercator

# bbox of dcsm model, xmin, ymin, xmax, ymax
bbox = [-15, 43, 13, 64]

# get a bounding box in web mercator
xymin = lat_lon_to_webmercator(bbox[1], bbox[0])
xymax = lat_lon_to_webmercator(bbox[3], bbox[2])

x_range = (xymin[0], xymax[0])
y_range = (xymin[1], xymax[1])


In [8]:
elevation = ds.get_data_lazy(xl=x_range, yl=y_range, max_pixel_size=1e3, source="local")

In [9]:
elevation.load()

<xarray.DataArray 'elevtn' (y: 6912, x: 5376)> Size: 297MB
array([[-5114., -5163., -5208., ...,  1282.,  1375.,  1446.],
       [-5099., -5149., -5197., ...,  1253.,  1342.,  1417.],
       [-5096., -5147., -5194., ...,  1228.,  1314.,  1391.],
       ...,
       [  -49.,   -50.,   -50., ...,   498.,   514.,   516.],
       [  -48.,   -49.,   -49., ...,   543.,   558.,   558.],
       [  -47.,   -48.,   -48., ...,   564.,   578.,   590.]],
      shape=(6912, 5376))
Coordinates:
  * x        (x) float64 43kB -1.722e+06 -1.721e+06 ... 1.565e+06 1.565e+06
  * y        (y) float64 55kB 5.166e+06 5.167e+06 ... 9.392e+06 9.392e+06
Attributes:
    crs:      EPSG:3857
    z_level:  8

In [ ]:
elevation.plot()

In [7]:
# Traditional way to get data, returns numpy arrays
x,y,z = ds.get_data(xl=x_range, yl=y_range, max_pixel_size=1e3)

In [ ]:
# New lazy way to get data, returns xarray DataArray with dask backend
elevation = ds.get_data_lazy(xl=x_range, yl=y_range, max_pixel_size=1e3)

In [ ]:
# now create an xarray data-arrat from the x,y coordinates with z as the values
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import colors

vmin = -50
vmax = 300
c_dem = plt.cm.terrain(np.linspace(0.25, 1, vmax))
if vmin < 0:
    c_bat = plt.cm.terrain(np.linspace(0, 0.17, max(1, abs(vmin))))
    c_dem = np.vstack((c_bat, c_dem))
cmap = colors.LinearSegmentedColormap.from_list("dem", c_dem)

fig,ax = plt.subplots(figsize=(10,10))
elevation.plot(ax=ax, cmap=cmap, vmin=vmin, vmax=vmax)


In [ ]:
# plot x,y,z data
xz, yz = np.meshgrid(x, y)

plt.figure(figsize=(10,10))
plt.pcolormesh(xz, yz, z, cmap=cmap, vmin=vmin, vmax=vmax)
plt.colorbar(label="Elevation (m)")
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.title("Elevation from GEBCO 2024")